In [1]:
!pip install --user uproot
!pip install --user tqdm
!pip install awkward
!pip install xgboost
!pip install sklearn

In [1]:
import uproot
import glob
import pandas as pd
from tqdm import tqdm
import numpy as np
import awkward as ak
import ROOT
import matplotlib.pyplot as plt

Welcome to JupyROOT 6.24/02


In [2]:
root_files=['WWG_emu','ST','TTGJets','ZGJets','WGJets','VV','tZq','fakeL','plj']
fdir='/home/pku/anying/cms/rootfiles/WWg/2018/'

In [3]:
f1 = uproot.open(fdir+'cutlep-out'+root_files[0]+'18.root')
tree=f1['Events']

In [5]:
necessary_cols=['lep1pt','lep1eta','lep1phi','lep2pt','lep2eta','lep2phi',
                'lep1_charge','lep2_charge','lep1_isprompt','lep2_isprompt',
                'lep1_is_tight','lep2_is_tight','n_loose_ele','n_loose_mu',
                'HLT_ee','HLT_mm','HLT_emu','ptll','mll','drll',
                'photonet','photoneta','photonphi','drl1a',
                'drl2a','mllg','PuppiMET_T1Smear_pt','PuppiMET_T1Smear_phi',
                'njets30','n_bjets20_medium','n_bjets20_loose',
                'scalef','ele_id_scale','ele_reco_scale','muon_id_scale','muon_iso_scale',
                'photon_id_scale','photon_veto_scale','btag_weight_medium'] 

In [ ]:
f = uproot.lazy(fdir+'cutlep-out'+root_files[0]+'18.root:Events')
events = ak.to_pandas(f[necessary_cols])

In [ ]:
import 

In [8]:
events.loc[events.index,'isSignal']=1

In [7]:
def get_df(root_file,year):
    f = uproot.lazy(fdir+'cutlep-out'+root_file+year+'.root:Events')
    events = ak.to_pandas(f[necessary_cols])
    if 'WWG_emu' in root_file:
        events.loc[events.index,'isSignal']=1
    else:
        events.loc[events.index,'isSignal']=1

,lep1pt,lep1eta,lep1phi,lep2pt,lep2eta,lep2phi,lep1_charge,lep2_charge,lep1_isprompt,lep2_isprompt,...,n_bjets20_medium,n_bjets20_loose,scalef,ele_id_scale,ele_reco_scale,muon_id_scale,muon_iso_scale,photon_id_scale,photon_veto_scale,btag_weight_medium
entry,,,,,,,,,,,,,,,,,,,,,
0,230.576904,2.337891,3.137695,99.569122,1.600342,-2.952148,-1,1,1,1,...,0,0,0.000373,0.979798,0.993902,0.965983,1.003635,1.000000,1.000000,0.992005
1,22.411518,0.553223,-1.636719,41.515800,1.194092,0.458679,-1,1,1,1,...,0,0,0.000373,0.966825,0.979101,0.993388,0.991155,1.000000,1.000000,0.990995
2,19.331579,-0.459229,-3.041992,52.686672,-1.635010,-0.549805,1,-1,1,1,...,0,0,0.000373,0.986532,0.988787,0.991589,0.985735,1.000000,1.000000,0.991306
3,110.908691,-0.326233,0.299438,53.624805,-0.037796,3.131836,1,-1,1,1,...,0,0,0.000373,0.970982,0.987692,0.991008,0.998273,0.970458,0.962842,0.990537
4,34.690849,-1.045410,-2.868164,24.554401,-1.539307,-1.619629,1,-1,1,1,...,0,0,-0.000373,1.000000,0.975439,0.984365,0.988747,1.000000,1.000000,0.995717
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43895,105.805023,-0.314941,0.315430,55.634136,-0.050056,-3.097168,1,-1,1,1,...,0,0,0.000373,0.970982,0.987692,0.991008,0.998273,1.000000,1.000000,0.993188
43896,32.725731,-1.087891,1.617676,34.899853,0.483093,0.436462,1,-1,1,1,...,0,0,0.000373,0.972900,0.986458,0.984365,0.988747,1.000000,1.000000,0.990984
43897,27.971409,1.838135,2.008789,52.443150,1.153564,0.630981,1,-1,1,1,...,0,0,0.000373,0.969388,0.980311,0.989815,0.999718,1.000000,1.000000,0.988893
